# COVID-19 Dashboard
Please find below the COVID-19 Dashboard.
The button below is able to update the data on demand to the latest data
The first graph is the graph of hospital beds and ITU beds used by COVID-19 patients over time.
Using the box on the left you can select to see either just the hospital beds, the ITU beds or both
The box on the right hand side allows you to change from a linear scale to a logarithmic one and vice versa.

In [82]:
# Importing the COVID API
from uk_covid19 import Cov19API
import json
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ipywidgets as wdg
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [92]:
# Creating a button to update data on demand
def access_api(button):
    apibutton.icon="check"
    apibutton.disabled=True
    

apibutton=wdg.Button(
    description='Update Graphs',
    disabled=False,
    button_style='info',
    tooltip='Click to update with latest PHE data',
    icon='refresh'
)
apibutton.on_click(access_api)

display(apibutton)


Button(button_style='info', description='Update Graphs', icon='refresh', style=ButtonStyle(), tooltip='Click t…

In [72]:
# Comparing Hospital Beds and ITU bed use over time
# creating the structure for the API

filters = [
    'areaType=overview' # taking data from the whole country
]
#taking the hospital beds and ITU beds per day
structure = { 
    "date": "date",
    "HospitalBeds": "hospitalCases",
    "ITUBeds" : "covidOccupiedMVBeds"
}
api = Cov19API(filters=filters, structure=structure)
hospanditu=api.get_json()
# writing the data out into a JSON file
with open("hospanditu.json", "wt") as OUTF:
    json.dump(hospanditu, OUTF) 

In [81]:
# wrangling the data into out DataFrame
with open("hospanditu.json", "rt") as INFILE:
    rawdata=json.load(INFILE)
# extracting the dates and getting a start and end date
datalist=rawdata['data']
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

# creating the dataframe filling in any missing dates
index=pd.date_range(startdate, enddate, freq='D')
hospitudf=pd.DataFrame(index=index, columns=['HospitalBeds', 'ITUBeds'])

# Filling the dataframe with data
for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['HospitalBeds', 'ITUBeds']:
        if pd.isna(hospitudf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            hospitudf.loc[date, column]=value

hospitudf.fillna(0.0, inplace=True)

# saving the data into a pickle file
hospitudf.to_pickle("hospitudf.pkl")


In [94]:
# Creating an interactive table:
hospitudf=pd.read_pickle("hospitudf.pkl")
dataselect=wdg.SelectMultiple(
    options=['HospitalBeds', 'ITUBeds'],
    value=['HospitalBeds', 'ITUBeds'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['Linear', 'Logarithmic'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([dataselect, scale])

def hospitu_graph(gcols, gscale):
    if gscale=='Linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        hospitudf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


graph=wdg.interactive_output(hospitu_graph, {'gcols': dataselect, 'gscale': scale})

display(controls, graph)



Output()